In [4]:
pip install tweepy

Note: you may need to restart the kernel to use updated packages.


In [5]:
import tweepy

#client = tweepy.Client(bearer_token='AAAAAAAAAAAAAAAAAAAAABnpcAEAAAAA%2FZ8s6rLM%2FzDX6roMgzKW4uRVLwY%3DziIAQs731EOnAN3l0Fbp0fFRH4eixgbFRqvyCVuKuK529uEqbh')

In [3]:
pip install ruamel-yaml

     |████████████████████████████████| 109 kB 945 kB/s eta 0:00:01
     |████████████████████████████████| 539 kB 1.4 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [5]:
# Replace with your own search query
query = 'from:suhemparack -is:retweet'

tweets = client.search_recent_tweets(query=query, tweet_fields=['context_annotations', 'created_at'], max_results=100)

for tweet in tweets.data:
    print(tweet.text)
    if len(tweet.context_annotations) > 0:
        print(tweet.context_annotations)

@chimeracoder Sorry to hear about your experience Aditya 😢 if there’s anything I can do to help going forward please lmk
Tweets to Twitter Circle are only available in the #TwitterAPI via User Context, not via the App-only authentication. https://t.co/q0tFGG7Zdl
[{'domain': {'id': '46', 'name': 'Brand Category', 'description': 'Categories within Brand Verticals that narrow down the scope of Brands'}, 'entity': {'id': '781974596752842752', 'name': 'Services'}}, {'domain': {'id': '47', 'name': 'Brand', 'description': 'Brands and Companies'}, 'entity': {'id': '10045225402', 'name': 'Twitter'}}]
My favorite time of the year is here 🎉

تقبل الله منا ومنكم

#EidMubarak https://t.co/52kH0IgFBW
[{'domain': {'id': '29', 'name': 'Events [Entity Service]', 'description': 'Entity Service related Events domain'}, 'entity': {'id': '1466236020010016770', 'name': 'Ramadan'}}]
This Repl on @Replit showcases how you can explore Tweet annotations in v2 using the sampled stream endpoint https://t.co/kuMh6

In [6]:
auth = tweepy.AppAuthHandler('7LAdt5hpLhHgqTLbjxq6OUew1', 'cbBkuL4lzRiG2GH5mK5WF4gr4prxYQnZDHHMLum63jACSgdRI0')
api = tweepy.API(auth)

In [7]:
places = api.search_geo(query="Colombia", granularity="country")

In [8]:
place_id = places[0].id

In [9]:
tweets = api.search_tweets(q="place:%s" % place_id)
for tweet in tweets:
    print(tweet.text + " | " + tweet.place.name if tweet.place else "Undefined place")

@cielo_rusinque @QuinteroCalle Siga soñando, una vez la @FiscaliaCol lo ponga preso se les cae esa estructurita de corrupción | Medellín
@fdbedout Si 🤡 periodista la gente esta verraca de tanto Hampón que quiere violentar las normas 👎🏼🤬… https://t.co/x3oCgdBsTi | Medellín
JAJAJAJAJAJAJAJA https://t.co/ZjbDMNC5yY | Barranquilla
🫠 https://t.co/f07XyZxokM | Antioquia
@Fabipinto21 Quien es esa? | Bello
@Nataliaescribe Y usted está haciendo el ridículo ahora | Medellín
Los pueblitos de Cundinamarca 🥶 | Sutatausa
@davidbarguil @FicoGutierrez @NoticiasCaracol No lo hicieron con duque mucho menos con el periquero. | Bogotá, D.C.
@RoyBarreras @petrogustavo Sera que tal compro al sistema electoral? Indra ya tendra todo listo? | Santa Rosa de Cabal
@AlejoGalvez11 Mmm no | Marinilla
@metrodemedellin Alguna vez se han puesto a analizar en por qué hace más calor en los metros nuevos que en los viejos, 🫣 | Medellín
Esta mezcla me encanta 😋 https://t.co/qWpGrezzIV | Palmira
Yo se que no hay que quejar